# population analysis on ephys units across sessions

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Niell Lab\.conda\envs\DLC-GPU2\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import sys
sys.path.insert(0, '/Users/Niell Lab/Documents/GitHub/FreelyMovingEphys/')

In [3]:
from util.paths import find
from project_analysis.ephys.ephys_utils import load_ephys
from project_analysis.ephys.population_utils import make_unit_summary, make_session_summary

## if the data are spread across directories for individual recordings, and need to be pooled

In [ ]:
# pool data listed in csv file
df = load_ephys(r'T:/freely_moving_ephys/batch_files/completed_experiment_pool.csv')

In [ ]:
df

In [ ]:
# save as one h5 file
df.to_hdf(r'T:\freely_moving_ephys\population_summaries\050321/exp_pool_all_data_050321.h5', 'w')

In [ ]:
# make a pdf summary
make_unit_summary(df, r'T:\freely_moving_ephys\population_summaries\050321')

## if there is already an h5 file of pooled data

In [4]:
# load existing .h5 file
df = pd.read_hdf(r'T:\freely_moving_ephys\population_summaries\050321/exp_pool_all_data_050321.h5')

In [ ]:
# make a pdf summary
# make_unit_summary(df, r'T:\freely_moving_ephys\population_summaries\050321')

In [5]:
make_session_summary(df, r'T:\freely_moving_ephys\population_summaries\050321')

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [05:58<00:00, 16.30s/it]

saving pdf
